In [105]:
import preprocessing as pp
import random
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import data
import pandas as pd

In [106]:
df=pd.read_csv('projtest.csv')

In [108]:
df['decision_binary'].value_counts()

0.0    100
1.0     17
Name: decision_binary, dtype: int64

In [109]:
100/117

0.8547008547008547

In [76]:
train, test, val, TEXT, LABEL = pp.get_data('projtrain.csv', 'projval.csv', 'projtest.csv', None)

Connected!


In [79]:
TEXT.build_vocab(train)
print(TEXT.vocab.itos[10]) 

evidence


In [130]:
# https://stackoverflow.com/questions/43567475/how-to-load-word-vector-model-binary-file-of-glove-cooccurence-bin
# https://github.com/DocketScience/LeGloVe

from glove import Glove
leglove_model = Glove.load('LeGlove.model')

# for leglove

# TEXT.build_vocab(train_data, 
#                 max_size = MAX_VOCAB_SIZE, 
#                 vectors = "glove.6B.100d", 
#                 unk_init = torch.Tensor.normal_)

ModuleNotFoundError: No module named 'Glove'

In [80]:
len(train)

714

In [81]:
len(test)

117

In [82]:
len(val)

59

In [83]:
LABEL.build_vocab(train)

In [84]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'petitioner', 'resident', 'cms', 'ex', 'facility', 'c.f.r', 'care', 'i.g']


In [86]:
len(TEXT.vocab)

32168

In [87]:
BATCH_SIZE = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, val, test), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.alj_text),
    sort_within_batch = True, 
    device = device)

In [88]:
def binary_accuracy(preds, y):
    """
    Return accuracy per batch
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [89]:


class WordEmbAvg(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, pad_idx, two_layers=True, dropout_p=0.0):
        
        super().__init__()
        
        # Define an embedding layer, a couple of linear layers, and 
        # the ReLU non-linearity.

        ##YOUR CODE HERE##
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        if two_layers == True:
            self.linear1 = nn.Linear(embedding_dim, hidden_dim)
            self.linear2 = nn.Linear(hidden_dim, output_dim) 
        else:
            self.linear1 = nn.Linear(embedding_dim, output_dim)
            self.linear2 = None
        self.relu = nn.ReLU()
        self.drop_layer = nn.Dropout(p=dropout_p)

        
        
    def forward(self, text):

        ##YOUR CODE HERE##
        embedded = self.embedding(text)
        embedded = embedded.mean(0)
        if not self.linear2:
            linear1_output = self.linear1(embedded)
            output = self.relu(linear1_output)
            output = self.drop_layer(output)
            return output
        else:
            linear1_output = self.linear1(embedded)
            linear2_input = self.relu(linear1_output)
            output = self.linear2(linear2_input)
            output = self.drop_layer(output)
            return output

In [132]:
class Training_module( ):

    def __init__(self, model):
        self.model = model
        self.loss_fn = nn.BCEWithLogitsLoss()
        ##YOUR CODE HERE##
        # Choose an optimizer. optim.Adam is a popular choice
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
    
    def train_epoch(self, iterator):
        '''
        Train the model for one epoch. For this repeat the following, 
        going through all training examples.
        1. Get the next batch of inputs from the iterator.
        2. Determine the predictions using a forward pass.
        3. Compute the loss.
        4. Compute gradients using a backward pass.
        5. Execute one step of the optimizer to update the model paramters.
        '''
        epoch_loss = 0
        epoch_acc = 0
    
        for batch in iterator:
          # batch.alj_text has the texts and batch.decision_binary has the labels.
        
            self.optimizer.zero_grad()
                
            ##YOUR CODE HERE##
            
            predictions = self.model(batch.alj_text).squeeze(1)
            loss = self.loss_fn(predictions, batch.decision_binary)
            accuracy = binary_accuracy(predictions, batch.decision_binary)
        
            loss.backward()
            self.optimizer.step()
            epoch_loss += loss.item()
            epoch_acc += accuracy.item()
        
        return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
    def train_model(self, train_iterator, dev_iterator, num_epochs=5):
        """
        Train the model for multiple epochs, and after each evaluate on the
        development set.  Return the best performing model.
        """  
        dev_accs = [0.]
        for epoch in range(num_epochs):
            self.train_epoch(train_iterator)
            dev_acc = self.evaluate(dev_iterator)
            print(f"Epoch {epoch}: Dev Accuracy: {dev_acc[1]} Dev Loss:{dev_acc[0]}")
            if dev_acc[1] > max(dev_accs):
                best_model = copy.deepcopy(self)
            dev_accs.append(dev_acc[1])
        return best_model.model
                
    def evaluate(self, iterator):
        '''
        Evaluate the performance of the model on the given examples.
        '''
        epoch_loss = 0
        epoch_acc = 0
        all_predicts = []
    
        with torch.no_grad():
    
            for batch in iterator:
                
                predictions = self.model(batch.alj_text).squeeze(1)
                all_predicts.append(predictions)
            
                loss = self.loss_fn(predictions, batch.decision_binary)
            
                acc = binary_accuracy(predictions, batch.decision_binary)
        
                epoch_loss += loss.item()
                epoch_acc += acc.item()
        #print(all_predicts)
        return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
    
        

In [133]:
INPUT_DIM = len(TEXT.vocab)
#You can try many different embedding dimensions. Common values are 20, 32, 64, 100, 128, 512
EMBEDDING_DIM = 100
HIDDEN_DIM = 50
OUTPUT_DIM = 1
#Get the index of the pad token using the stoi function
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [134]:
INPUT_DIM

32168

In [135]:
EMBEDDING_DIM

100

In [136]:
HIDDEN_DIM

50

In [137]:
OUTPUT_DIM

1

In [138]:
PAD_IDX

1

In [139]:
for batch in train_iterator:
    print(batch)
    print(batch.alj_text, batch.decision_binary)


[torchtext.data.batch.Batch of size 10]
	[.alj_text]:[torch.LongTensor of size 3570x10]
	[.decision_binary]:[torch.FloatTensor of size 10]
tensor([[ 262,  279,  329,  ...,  262,  262,  262],
        [  35, 4519,   62,  ...,   35,   35,   35],
        [ 339, 3210,  417,  ...,  339,  339,  339],
        ...,
        [ 154,    3,  176,  ...,    1,    1,    1],
        [  48, 1574,    1,  ...,    1,    1,    1],
        [ 176,    1,    1,  ...,    1,    1,    1]]) tensor([0., 1., 1., 0., 0., 0., 0., 0., 1., 0.])

[torchtext.data.batch.Batch of size 10]
	[.alj_text]:[torch.LongTensor of size 2269x10]
	[.decision_binary]:[torch.FloatTensor of size 10]
tensor([[ 279,  279,  279,  ...,  262,  279,  279],
        [9977, 7658, 6422,  ...,   35, 6015, 1737],
        [9653,  102, 5167,  ...,  339, 1270, 1072],
        ...,
        [ 154,    1,    1,  ...,    1,    1,    1],
        [  48,    1,    1,  ...,    1,    1,    1],
        [ 176,    1,    1,  ...,    1,    1,    1]]) tensor([1., 1., 0.,


[torchtext.data.batch.Batch of size 10]
	[.alj_text]:[torch.LongTensor of size 1342x10]
	[.decision_binary]:[torch.FloatTensor of size 10]
tensor([[262, 262, 262,  ..., 262, 262, 262],
        [ 35,  35,  35,  ...,  35,  35,  35],
        [339, 339, 339,  ..., 339, 339, 339],
        ...,
        [154,   1,   1,  ...,   1,   1,   1],
        [ 48,   1,   1,  ...,   1,   1,   1],
        [176,   1,   1,  ...,   1,   1,   1]]) tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

[torchtext.data.batch.Batch of size 10]
	[.alj_text]:[torch.LongTensor of size 820x10]
	[.decision_binary]:[torch.FloatTensor of size 10]
tensor([[  262,   262,   279,  ...,   262,   262,   262],
        [   35,    35,  4488,  ...,    35,    35,    35],
        [  339,   339, 11020,  ...,   339,   339,   339],
        ...,
        [  154,     1,     1,  ...,     1,     1,     1],
        [   48,     1,     1,  ...,     1,     1,     1],
        [  176,     1,     1,  ...,     1,     1,     1]]) tensor([0., 0., 0., 


[torchtext.data.batch.Batch of size 10]
	[.alj_text]:[torch.LongTensor of size 3917x10]
	[.decision_binary]:[torch.FloatTensor of size 10]
tensor([[ 262,  279,  262,  ...,  262,  262,  262],
        [  35, 2158,   35,  ...,   35,   35,   35],
        [ 339, 1449,  339,  ...,  339,  339,  339],
        ...,
        [ 154,    1,    1,  ...,    1,    1,    1],
        [  48,    1,    1,  ...,    1,    1,    1],
        [ 176,    1,    1,  ...,    1,    1,    1]]) tensor([0., 0., 0., 0., 1., 0., 0., 0., 1., 0.])

[torchtext.data.batch.Batch of size 10]
	[.alj_text]:[torch.LongTensor of size 2546x10]
	[.decision_binary]:[torch.FloatTensor of size 10]
tensor([[ 279,  262,  262,  ...,  279,  262,  262],
        [7323,   35,   35,  ..., 6235,   35,   35],
        [  76,  339,  339,  ..., 1449,  339,  339],
        ...,
        [  33,    1,    1,  ...,    1,    1,    1],
        [ 129,    1,    1,  ...,    1,    1,    1],
        [ 207,    1,    1,  ...,    1,    1,    1]]) tensor([0., 1., 0.,


[torchtext.data.batch.Batch of size 10]
	[.alj_text]:[torch.LongTensor of size 1119x10]
	[.decision_binary]:[torch.FloatTensor of size 10]
tensor([[262, 262, 262,  ..., 262, 262, 262],
        [ 35,  35,  35,  ...,  35,  35,  35],
        [339, 339, 339,  ..., 339, 339, 339],
        ...,
        [154, 176,   1,  ...,   1,   1,   1],
        [ 48,   1,   1,  ...,   1,   1,   1],
        [176,   1,   1,  ...,   1,   1,   1]]) tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1.])

[torchtext.data.batch.Batch of size 10]
	[.alj_text]:[torch.LongTensor of size 1020x10]
	[.decision_binary]:[torch.FloatTensor of size 10]
tensor([[262, 262, 262,  ..., 262, 262, 262],
        [ 35,  35,  35,  ...,  35,  35,  35],
        [339, 339, 339,  ..., 339, 339, 339],
        ...,
        [154,   1,   1,  ...,   1,   1,   1],
        [ 48,   1,   1,  ...,   1,   1,   1],
        [176,   1,   1,  ...,   1,   1,   1]]) tensor([1., 1., 1., 0., 1., 0., 0., 1., 0., 0.])


In [140]:
model = WordEmbAvg(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX)

In [141]:
model = model.to(device)
tm = Training_module(model)

#Training the model
best_model = tm.train_model(train_iterator, valid_iterator)

Epoch 0: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5260844975709915
Epoch 1: Dev Accuracy: 0.8611111144224802 Dev Loss:0.49399032195409137
Epoch 2: Dev Accuracy: 0.8444444437821707 Dev Loss:0.48934957136710483
Epoch 3: Dev Accuracy: 0.7944444417953491 Dev Loss:0.499497930208842
Epoch 4: Dev Accuracy: 0.7444444497426351 Dev Loss:0.5466504593690237


In [142]:
tm.model = best_model
test_loss, test_acc = tm.evaluate(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.512 | Test Acc: 84.40%


In [143]:
norms = torch.norm(best_model.embedding.weight, p=2, dim=1, keepdim=True).data.squeeze()
norms

tensor([ 9.4312, 10.8774,  8.9406,  ...,  9.8766,  8.7714,  9.7347])

In [144]:
highest_10 = norms.argsort()[-10:]

In [145]:
for i in highest_10:
    print(TEXT.vocab.itos[i])

uppergastrointestinal
fluorescent
dwight
426.425(c)(1
405.874(c)(5
p-12
overrule
fore
require
cr2171


In [146]:
model = WordEmbAvg(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX)
model = model.to(device)
tm = Training_module(model)
tm.optimizer = optim.Adagrad(tm.model.parameters(), lr=1e-3)
optim_model = tm.train_model(train_iterator, valid_iterator)
tm.model = optim_model
test_loss, test_acc = tm.evaluate(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Epoch 0: Dev Accuracy: 0.8611111144224802 Dev Loss:0.6384853621323904
Epoch 1: Dev Accuracy: 0.8611111144224802 Dev Loss:0.6164842446645101
Epoch 2: Dev Accuracy: 0.8611111144224802 Dev Loss:0.6005518039067587
Epoch 3: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5879337390263876
Epoch 4: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5775904506444931
Test Loss: 0.646 | Test Acc: 84.40%


In [147]:
model = WordEmbAvg(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX, two_layers=False)
model = model.to(device)
tm = Training_module(model)
onelayer_model = tm.train_model(train_iterator, valid_iterator)
tm.model = onelayer_model
test_loss, test_acc = tm.evaluate(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Epoch 0: Dev Accuracy: 0.8444444437821707 Dev Loss:0.6937061349550883
Epoch 1: Dev Accuracy: 0.8444444437821707 Dev Loss:0.6936950087547302
Epoch 2: Dev Accuracy: 0.8444444437821707 Dev Loss:0.6937471727530161
Epoch 3: Dev Accuracy: 0.8444444437821707 Dev Loss:0.6939022243022919
Epoch 4: Dev Accuracy: 0.8444444437821707 Dev Loss:0.6940129399299622
Test Loss: 0.693 | Test Acc: 81.90%


In [148]:
for dropout_rate in [0, 0.1, 0.25, 0.5, 0.75]:
    print(f'Dropout rate: {dropout_rate}')
    model = WordEmbAvg(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX, dropout_p=dropout_rate)
    model = model.to(device)
    tm = Training_module(model)
    dropout_model = tm.train_model(train_iterator, valid_iterator)
    tm.model = dropout_model
    test_loss, test_acc = tm.evaluate(test_iterator)
    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Dropout rate: 0
Epoch 0: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5628097454706827
Epoch 1: Dev Accuracy: 0.8611111144224802 Dev Loss:0.49027171234289807
Epoch 2: Dev Accuracy: 0.8444444437821707 Dev Loss:0.49950480461120605
Epoch 3: Dev Accuracy: 0.7944444417953491 Dev Loss:0.4926765412092209
Epoch 4: Dev Accuracy: 0.7444444497426351 Dev Loss:0.5308692256609598
Test Loss: 0.573 | Test Acc: 84.40%
Dropout rate: 0.1
Epoch 0: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5035866896311442
Epoch 1: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5168760567903519
Epoch 2: Dev Accuracy: 0.8611111144224802 Dev Loss:0.505170946319898
Epoch 3: Dev Accuracy: 0.8611111144224802 Dev Loss:0.4701434224843979
Epoch 4: Dev Accuracy: 0.8444444437821707 Dev Loss:0.5218525975942612
Test Loss: 0.537 | Test Acc: 84.40%
Dropout rate: 0.25
Epoch 0: Dev Accuracy: 0.8611111144224802 Dev Loss:0.531109074751536
Epoch 1: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5939091295003891
Epoch 2: Dev Accuracy: 0.8611111144

In [149]:
df = pd.DataFrame(columns=['embedding', 'hidden', 'accuracy'])
for embed_dimension in [20, 32, 64, 100]:
    for hidden_dimension in [10, 25, 40, 50]:
        print(f'Embedding dimensions: {embed_dimension}, Hidden dimensions: {hidden_dimension}')
        model = WordEmbAvg(INPUT_DIM, embed_dimension, hidden_dimension, OUTPUT_DIM, PAD_IDX)
        model = model.to(device)
        tm = Training_module(model)
        dim_model = tm.train_model(train_iterator, valid_iterator)
        tm.model = dim_model
        test_loss, test_acc = tm.evaluate(test_iterator)
        print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
        row = [embed_dimension, hidden_dimension, test_loss*100]
        #row = [embed_dimension, hidden_dimension, test_acc*100]
        df.loc[len(df)] = row

Embedding dimensions: 20, Hidden dimensions: 10
Epoch 0: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5548453231652578
Epoch 1: Dev Accuracy: 0.8611111144224802 Dev Loss:0.522187352180481
Epoch 2: Dev Accuracy: 0.8611111144224802 Dev Loss:0.4997718979914983
Epoch 3: Dev Accuracy: 0.8611111144224802 Dev Loss:0.4898165613412857
Epoch 4: Dev Accuracy: 0.8611111144224802 Dev Loss:0.4761844724416733
Test Loss: 0.562 | Test Acc: 84.40%
Embedding dimensions: 20, Hidden dimensions: 25
Epoch 0: Dev Accuracy: 0.8444444437821707 Dev Loss:0.6399096151192983
Epoch 1: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5413826902707418
Epoch 2: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5082739442586899
Epoch 3: Dev Accuracy: 0.8611111144224802 Dev Loss:0.49478266139825183
Epoch 4: Dev Accuracy: 0.8611111144224802 Dev Loss:0.4860636691252391
Test Loss: 0.554 | Test Acc: 84.40%
Embedding dimensions: 20, Hidden dimensions: 40
Epoch 0: Dev Accuracy: 0.8611111144224802 Dev Loss:0.5809823075930277
Epoch 1: Dev A

In [150]:
df

,embedding,hidden,accuracy
0,20.0,10.0,56.247431
1,20.0,25.0,55.409171
2,20.0,40.0,59.124228
3,20.0,50.0,59.612226
4,32.0,10.0,57.804537
5,32.0,25.0,58.998898
6,32.0,40.0,57.403537
7,32.0,50.0,55.301298
8,64.0,10.0,53.735669
9,64.0,25.0,61.885237
